# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,8.94,55,75,0.00,US,1696197356
1,1,waitangi,-43.9535,-176.5597,14.34,87,79,3.58,NZ,1696197357
2,2,coquimbo,-29.9533,-71.3436,14.85,77,100,3.60,CL,1696196966
3,3,aykhal,66.0000,111.5000,0.04,98,100,7.15,RU,1696197357
4,4,ilulissat,69.2167,-51.1000,-1.99,74,8,0.00,GL,1696197358


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points("Lat", 
                                        "Lng", 
                                        geo = True, 
                                        color = "City", 
                                        alpha=0.5, 
                                        size = "Humidity")

# Display the map
map_cities

:Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [80]:
# Narrow down cities that fit criteria and drop any results with null values
    #My ideal weather condition is 
    #   -temperatures between 20 to 25 fahrenheit.
    #   -temperatures cloudiness up to 2.
    #   -temperatures wind speed up to 5.
    #   -temperatures humidity up to 70.

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) 
                                    & (city_data_df["Max Temp"] >= 20)
                                    & (city_data_df["Cloudiness"] <= 2) 
                                    & (city_data_df["Wind Speed"] <= 5) 
                                    & (city_data_df["Humidity"] <= 80)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
262,262,jamestown,42.0970,-79.2353,23.62,61,0,4.12,US,1696197456
410,410,emerald,-23.5333,148.1667,20.75,63,0,4.81,AU,1696197753
476,476,ravar,31.2656,56.8055,20.82,19,0,1.64,IR,1696197777
549,549,al hazm,16.1641,44.7769,24.67,16,0,0.80,YE,1696197806


### Step 3: Create a new DataFrame called `hotel_df`.

In [81]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
262,jamestown,US,42.0970,-79.2353,61,
410,emerald,AU,-23.5333,148.1667,63,
476,ravar,IR,31.2656,56.8055,19,
549,al hazm,YE,16.1641,44.7769,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [82]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel","apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: No hotel found
emerald - nearest hotel: No hotel found
ravar - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
262,jamestown,US,42.0970,-79.2353,61,No hotel found
410,emerald,AU,-23.5333,148.1667,63,No hotel found
476,ravar,IR,31.2656,56.8055,19,No hotel found
549,al hazm,YE,16.1641,44.7769,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [83]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points("Lat", 
                                    "Lng", 
                                    geo = True, 
                                    color = "City", 
                                    alpha=0.5, 
                                    size = "Humidity",
                                    hover_cols=["Hotel Name", "Country"])

# Display the map
map_hotels

:Points   [Lat,Lng]   (City,Humidity,Hotel Name,Country)